In [18]:
# Procesamiento de datos
import pandas as pd
# Funciones algebráicas
import numpy as np
# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
## Estilos
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')
# Contador
from collections import Counter
# Métodos de iteración
from itertools import product
# Machine Learning
## Cross Validation
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
## Separación de datos de entrenamiento
from sklearn.model_selection import train_test_split
## Preprocesamiento de variables
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder, RobustScaler, MaxAbsScaler, QuantileTransformer, PowerTransformer
# Modelos de clasificacion
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

## Modelo de máquina de soporte vectorial
from sklearn.svm import SVC
## Clasificador de árbol de decisión
from sklearn.tree import DecisionTreeClassifier
## Clasificador de bosques aleatorios
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
## Naive Bayes
from sklearn.naive_bayes import GaussianNB
## Regresión Logística
from sklearn.linear_model import LogisticRegression
## Muestreado para balancear datos
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC
## Métricas
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc, roc_auc_score, ConfusionMatrixDisplay, f1_score, make_scorer, precision_score, recall_score, log_loss
## Ajuste de hiperparámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# módulo que proporciona soporte en tiempo de ejecución para sugerencias de tipo
from sklearn import datasets
from typing import Dict, Iterable, Any
# Desactivar los avisos
import warnings
warnings.filterwarnings('ignore')
# Importar los datos de yahoo
import yfinance as yf
# Librería para plotear mejor datos financieros
#import cufflinks as cf
#cf.set_config_file(offline = True)

He intentado escoger acciones con poca correlación. La explicación de ellas es:

AAPL: Apple Inc., uno de los fabricantes más grandes de tecnología de consumo, conocido por productos como el iPhone, el iPad y las MacBooks.

RELIANCE.NS: Reliance Industries Limited, una conglomerado multinacional indio que tiene negocios en diversos sectores como energía, petroquímicos, textiles, recursos naturales, y telecomunicaciones. El sufijo ".NS" indica que la acción se negocia en la National Stock Exchange of India.

BHP.AX: BHP Group Limited (anteriormente BHP Billiton), una de las compañías mineras y de recursos más grandes del mundo. El sufijo ".AX" indica que la acción se negocia en la Australian Securities Exchange.

SAN.MC: Banco Santander, S.A., un banco multinacional español y una de las mayores instituciones bancarias de la zona euro. El sufijo ".MC" indica que la acción se negocia en la Bolsa de Madrid.

REP.MC: Repsol, S.A., una empresa energética y petroquímica española. Al igual que SAN.MC, el sufijo ".MC" indica que la acción se negocia en la Bolsa de Madrid.

IBE.MC: Iberdrola, S.A., una empresa española de energía, una de las mayores productoras de energía renovable. También cotiza en la Bolsa de Madrid.

TEF.MC: Telefónica, S.A., una compañía multinacional de telecomunicaciones con sede en España. También cotiza en la Bolsa de Madrid.

Acciones con las que trabajaré: 

["AAPL", "RELIANCE.NS", "BHP.AX", "SAN.MC", "REP.MC", "IBE.MC", "TEF.MC"]

In [19]:
df = pd.read_csv("acciones_preprocesado.csv")
# quito los que devuelve 0 la dirección ya que no me calcula bien las métricas de los modelos
df = df[df.direction != 0]
df = df.sort_values(by="date")
df.reset_index(inplace=True, drop=True)


Lo primero que voy a hacer es transformar la fecha para poder meterla como variable numérica 

Ahora voy a separar los data frames en:
1.- variables categóricas para hacerle onehotencoding
2.- variables numéricas para aplicarle escaladores
3.- variables seno y cosenoidales que no se transforman ya más
4.- variable objetivo (dirección)

In [20]:
df_cat = df[["accion"]].copy()
columns_to_copy = [col for col in df.columns if col.startswith('sin') or col.startswith('cos')]
df_sincos = df[columns_to_copy].copy()
columns_to_exclude = columns_to_copy + ["accion", "direction", "price", "date"]
columns_to_select = [col for col in df.columns if col not in columns_to_exclude]
df_num = df[columns_to_select].copy()
df_dir = df[["direction"]].copy()

Empiezo haciendo one hot encoding sobre la única variable tipo texto que tengo. El nombre de la acción

In [21]:
# drop_first para evitar la colinealidad
df_cat = pd.get_dummies(df_cat, columns=["accion"], drop_first = False).astype(int)
df_cat 


,accion_AAPL,accion_IBE.MC,accion_REP.MC,accion_SAN.MC,accion_TEF.MC
0,0,0,1,0,0
1,0,1,0,0,0
2,0,0,0,1,0
3,0,0,0,0,1
4,0,0,0,1,0
...,...,...,...,...,...
16674,1,0,0,0,0
16675,0,1,0,0,0
16676,0,0,0,1,0
16677,0,0,1,0,0


Ahora aplico escaladores sobre las variables numéricas

Ahora ya tengo el df de categóricas con onehotencoding y las numéricas con los datos escalados. Por tanto los quiero volver a unir y juntar también con las variables seno y cosenoidales. Y tener solo separada la variable objetivo

In [22]:
df_num[df_num.isna().any(axis=1)]

,returns,price_lag1,high_lag1,low_lag1,lag1,lag2,lag3,lag4,lag5,sma200,sma_lower200,sma_upper200,ema_s,ema_l,roll_low,roll_high,year,day_of_year,week_of_year


In [23]:
# por otra parte me guardo la variable objetivo en este dataframe y veo que está balanceado pero que tiene algunos 0, 
# no se bien que hacer con ellos. Significa que la diferencia de precio de un día a otro ha sido 0
df_dir.direction.value_counts()

direction
 1.0    8506
-1.0    8173
Name: count, dtype: int64

Ahora vamos a entrenar un modelo solo para ver que tal funciona, si me das el ok Alejandro intento meterle un bucle para que pruebe distintos modelos y distintos escaladores. De momento hago la prueba con logistic regression.

In [24]:
df_num


,returns,price_lag1,high_lag1,low_lag1,lag1,lag2,lag3,lag4,lag5,sma200,sma_lower200,sma_upper200,ema_s,ema_l,roll_low,roll_high,year,day_of_year,week_of_year
0,0.004088,8.289960,19.885000,19.420000,-0.017008,0.004541,-0.003785,-0.006778,0.006023,7.438594,6.458718,8.418470,8.387049,8.272826,18.725000,20.530001,2010,293,42
1,-0.001024,3.037537,5.907000,5.824000,-0.001534,0.003754,0.006519,0.000517,0.016845,2.912467,2.486591,3.338342,2.991741,2.959894,5.500000,5.907000,2010,293,42
2,-0.000624,4.178141,9.111239,8.860599,0.008142,0.006414,0.007836,-0.006384,0.030827,4.051679,3.414417,4.688942,4.088681,4.083700,8.312903,9.111239,2010,293,42
3,-0.000258,7.719537,19.274195,19.056686,-0.003860,0.001028,0.000000,0.005671,0.023014,6.726953,5.912897,7.541010,7.578370,7.419077,17.672543,19.407665,2010,293,42
4,-0.004483,4.175536,8.957142,8.790048,-0.000624,0.008142,0.006414,0.007836,-0.006384,4.048097,3.421868,4.674325,4.102043,4.090503,8.312903,9.111239,2010,294,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16674,-0.005439,193.333298,194.660004,193.169998,0.002224,0.000518,-0.002845,-0.005563,-0.000770,178.699130,156.798249,200.600011,193.919339,192.080463,191.089996,199.619995,2023,363,52
16675,0.003798,11.625863,11.910000,11.820000,-0.000845,-0.002110,0.006344,0.002124,-0.005512,10.910265,10.082776,11.737753,11.555821,11.358391,11.675000,12.015000,2023,363,52
16676,-0.002114,3.713319,3.817500,3.775000,-0.005923,0.000525,0.003947,0.001055,-0.000923,3.337289,2.873884,3.800693,3.738733,3.715193,3.735500,3.966500,2023,363,52
16677,-0.001486,13.072656,13.750000,13.465000,-0.022753,0.010578,0.001101,-0.005126,0.008804,13.331832,11.812746,14.850918,13.228394,13.333067,13.160000,14.010000,2023,363,52


In [25]:
scalers = {
    "MinMax Scaler": MinMaxScaler(),
    "Standard Scaler": StandardScaler(),
    "MaxAbs Scaler": MaxAbsScaler(),
    "Robust Scaler": RobustScaler(),
    "Quant-Normal": QuantileTransformer(output_distribution="normal"),
    "Quant-Uniform": QuantileTransformer(output_distribution="uniform"),
    "PowerTransf-YeoJohnson": PowerTransformer(method='yeo-johnson')
}


# Define classification models
models = {
    "SVC": SVC(gamma="auto", C=10000),
    "Logistic Regression": LogisticRegression(penalty="l2"),
    "KNeighbors Classifier": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree Classifier": DecisionTreeClassifier(),
    "GaussianNB": GaussianNB(),
    "Random Forest Classifier": RandomForestClassifier(n_estimators=100),
    "MLP": MLPClassifier(tol=0.05, max_iter=1000),
    #"Gaussian Process Classifier": GaussianProcessClassifier(), # tarda mucho este? comprobar
    "Ada Boost Classifier": AdaBoostClassifier(n_estimators=100),
    "Bagging Classifier": BaggingClassifier(estimator=SVC(), n_estimators=10)
}

In [26]:

# Configurar el método de validación cruzada
cv = TimeSeriesSplit(n_splits=4)

# Diccionario de métricas de scoring
scoring = {
    'accuracy': 'accuracy',
    #'precision': 'precision',
    'recall': 'recall',
    #'f1': 'f1',
    #'roc_auc': 'roc_auc'
}

# DataFrame para guardar los resultados finales
results = pd.DataFrame()
df_dir_reduc = df_dir[-2000:].copy()
# Bucle para cada escalador
for scaler_name, scaler in scalers.items():
    # Aplicar el escalador a los datos
    X_scaled = pd.DataFrame(scaler.fit_transform(df_num.copy()), columns = df_num.columns)  
    df_variables = pd.concat([df_cat, df_sincos, X_scaled], axis=1)
    df_variables_reduc = df_variables[-2000:].copy()
    for model_name, model in models.items():
        # Ejecutar cross_validate
        cv_results = cross_validate(model, df_variables_reduc, df_dir_reduc, cv=cv, scoring=scoring)
        mean_accuracy = cv_results['test_accuracy'].mean()
        # Guardar los resultados en el DataFrame
        temp_df = pd.DataFrame({
            'Scaler': [scaler_name] * cv.get_n_splits(),
            'Model': [model_name] * cv.get_n_splits(),
            'Accuracy': cv_results['test_accuracy'],
            #'Precision': cv_results['test_precision'],
            'Recall': cv_results['test_recall'],
            #'F1-Score': cv_results['test_f1'],
            #'ROC AUC': cv_results['test_roc_auc']
        })
        
        # Concatenar el resultado al DataFrame de resultados
        results = pd.concat([results, temp_df], ignore_index=True)

In [27]:
# Supongo que 'results' ya está definido anteriormente y contiene los datos necesarios.

# Crear la tabla pivotada
results_pivot = results.pivot_table(index=['Scaler'], columns=['Model'], values=['Accuracy', 'Recall'], aggfunc='mean')

# Reestructurar la tabla con las métricas como columnas
results_pivot_reversed = results_pivot.reset_index().melt(id_vars=['Scaler'], var_name=['Metric', 'Model'])

# Crear una tabla pivote con las métricas y los modelos

results_pivot_final = results_pivot_reversed.pivot_table(index='Scaler', columns=['Model', 'Metric'], values='value')

# Asegurarse de que las columnas estén correctamente ordenadas y jerárquicas
results_pivot_final = results_pivot_final.stack(level=0).unstack(level=1).reorder_levels([1, 0], axis=1).sort_index(axis=1, level=0)

# Función para resaltar el máximo valor en cada columna a partir de la fila 4
def highlight_max(data):
    attr = 'background-color: lightgreen'
    is_max = data.apply(lambda x: x == x.max(), axis=0)
    return is_max.applymap(lambda x: attr if x else '')

# Aplicar el estilo
results_pivot_final_styled = results_pivot_final.style.apply(highlight_max, axis=None)

# Exportar a Excel
results_pivot_final_styled.to_excel('metricas_modelos_clasificacion_look_ahead_bias.xlsx', engine='openpyxl')

print("El archivo ha sido exportado exitosamente con los valores máximos resaltados.")


El archivo ha sido exportado exitosamente con los valores máximos resaltados.
